Данное домашнее задание посвящено схеме базе данных, связанной с футбольным командами. Схема расположена в hw_diagram.jpg
## Описание таблиц 

### Positions 
Позиции игроков
id - уникальный идентификатор позиции 
Name - Название таблицы 

### Coaches 
Таблица тренеров 
id - уникальный идентификатор тренера
Name - имя тренера 

## Teams 
Таблица команд 
id - уникальный идентификатор  команды 
Name - имя команды 
Location - расположение команды 
Coach_id - идентификатор тренера 

### Players 
Таблица игроков 
Id - уникальный идентификатор игрока 
Name - имя игрока 
Age - возраст игрока 
Position_id - идентификатор позиции 
Team_id - идентификатор команды 
Start_date - дата начала игры в команде 

### PlayersArchive 
Архив по игрокам (информация о том, в какой команде игрок играл ранее) 
player_id - идентификатор игрока 
team_id - идентификатор команды 
start_date - дата начала игры 
end_date - дата окончания игры

# Задание 1
Реализовать перенакатываемые sql-скрипты по схеме hw_diagram, приложенного ниже.

In [1]:
%load_ext sql
%sql sqlite:///HomeW.db

'Connected: @HomeW.db'

In [62]:
%%sql
PRAGMA foreign_keys=ON;
drop table if exists PlayersArchive;
drop table if exists Players;
drop table if exists Teams;
drop table if exists Coaches;
drop table if exists Positions;


create table Coaches (
    id integer not null primary key,
    name varchar(30)
);
create table Positions(
    id integer not null primary key,
    name varchar(30)
);
create table Teams(
    id integer not null primary key,
    name varchar(30),
    location varchar(30),
    coach_id integer,
    foreign key(coach_id) references Coaches(id)
);
create table Players(
    id integer not null primary key,
    name varchar(30),
    age integer,
    position_id integer,
    team_id integer,
    start_date date,
    foreign key(position_id) references Positions(id)
);

create table PlayersArchive(
    player_id integer,
    team_id integer,
    start_date date,
    end_date date,
    foreign key(player_id) references Players(id),
    foreign key(team_id) references Teams(id)
);


 * sqlite:///HomeW.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

# Задание 2
Написать sql-скрипт, осуществляюший добавление всех возможных позиций игроков. (В данной задаче можно ограничится четырьмя - вратарь, защитник, полузащитник, нападающий)

In [31]:
%%sql
insert into Positions (name) values ("Goalkeeper"),("Full-back"),("Wing-back"),("Forward")

 * sqlite:///HomeW.db
4 rows affected.


[]

# Задание 3
Написать функцию, осуществляющую добавление тренера. На вход функции подается имя тренера. При невозможности вставки функция должна обработать данное исключение и вывести пользователю информацию о проблеме

После написания функции необходимо добавить несколько тренеров для таблицы.

In [10]:
import sqlite3
db=sqlite3.connect('HomeW.db')

In [65]:
def add_coach(name):
    cursor = db.cursor()
    try:
        cursor.execute( "insert into Coaches(name) values (?)",(name,))
    except sqlite3.Error as e:
        print(e.args[0])
    db.commit()

In [66]:
add_coach("Coach_1")
add_coach("Coach_2")
add_coach("Coach_3")

In [81]:
%%sql
select * from Teams

 * sqlite:///HomeW.db
Done.


id,name,location,coach_id
1,team_1,Moscow,1


# Задание 4

Написать функцию, осуществляющую добавление команды. На вход функция принимает (имя команды, местоположение, имя тренера).
На выходе должна быть добавлена команда.

После написания функции необходимо добавить несколько команд

In [79]:
def add_team(name,location,coach_name):
    cursor = db.cursor()
    query = "insert into Teams(name,location,coach_id) values('{0}','{1}',(select id from Coaches where name = '{2}'))".format(name,location,coach_name)
    cursor.execute(query)
    db.commit()

In [82]:
add_team("team_1","Moscow","Coach_1")
add_team("team_2","Moscow","Coach_2")

# Задание 5

Написать функцию, осуществляющую обновление информации о команде. У функции на входе должен быть один обязательный параметр - название команды и два необязательных параметра - имя тренера и местоположение. При этом хотя бы один из необязательных параметров должен быть заполнен. Функция должна обновлять значение переданных необязательных параметров.

После написания функции вызовите данную функцию несколько раз

# Задание 6

Написать функцию, осуществляющую добавление нового футболиста в таблицу игроков. На входе (имя, возраст, название позиции, название команды, дата перехода в команду(необязательный параметр)). Если дата перехода не указана, то нужно использовать текущую дату. Предусмотреть различные ошибки (такие как неверное имя позиции, неверное название команды, возраст < 10 и > 50 и т.п.)

После написания функции необходимо добавить несколько игроков.

# Задание 7
Написать функцию для перевода игрока в другую команду. Данная функция должна осуществить архивирование текущей записи в таблице игроков в таблицу players_archive с заполнение end_date - датой начала игры за новую команду - 1 день. А также осуществлять изменение поля в таблице Players. При возникновении ошибки должен осуществляться полный откат транзакции.